# Workbench for ResumeAssistant

In [13]:
from pdf2image import convert_from_path
# ImageWorker takes in the path to jpg resume image and returns the contents of the resume in a json format. 
from resumeassistant.operator.worker import ImageWorker, TextWorker
from resumeassistant.data.candidate_record import Record
from openai import OpenAI
import json

# PDF to jpg image converter function
This function takes the input path for pdf-resume and output path for resume image. The output images are stored in 'jpg' format. 

In [23]:
def get_img(pdf_path, img_path):
      # Method for converting pdf resumes to jpg images. 
      images = convert_from_path(pdf_path)
      # print(type(images))
      for i in range(len(images)):
        
            # Save pages as images in the pdf
          images[i].save(img_path + str(i) +'.jpg', 'JPEG')

This is the code to extract OpenAI key stored in `gpt_key.json` file. \
Format: \
    ```{ 
	"api_key": \<open_ai_key\>
    }```

In [24]:
with open('gpt_key.json', 'r') as file:
      data = json.load(file)
      api_key = data["api_key"]

## Resume Parsing
This is the code for converting the PDF resume to an image and parsing it using "GPT-4o". 

In [25]:
client = OpenAI(api_key=api_key)
get_img('Harish_Resume_PhD.pdf', './Harish_Resume')

sys_prompt = "You are a resume parser. Given the image of the resume, parse every section and provide the output in a JSON format."
user_prompt="Can you parse all sections of this resume?"

image_worker = ImageWorker(client=client, model_id="gpt-4o-mini", sys_prompt=sys_prompt, user_prompt=user_prompt)
parsed_resume = image_worker.get_output('Harish_Resume0.jpg')
print(parsed_resume)

{'name': 'Harish Sista', 'contact_info': {'phone': '(201) 310-5683', 'email': 'hsista@stevens.edu', 'address': '2642 28th St, Apt 4F, Astoria, NY-11102'}, 'summary': 'Ph.D. Candidate researching in ML and NLP with 7+ years of experience working with ML models, Big Data, and Cloud Computing platforms and 2+ years of experience working in a professional environment. Spearheaded an Evidence Extraction and Fact Verification model using Prompt Engineering and tested it with various LLMs. Invented a Black Box Optimization model for online hyperparameter fine-tuning, significantly enhancing model performance in real-time applications.', 'skills': {'programming_languages': ['Python', 'SQL', 'C++', 'Java', 'Swift', 'HTML'], 'ai_tools': ['PyTorch', 'HuggingFace', 'TensorFlow', 'AutoML', 'SciKitLearn', 'NumPy'], 'llms': ['GPT', 'LLAMA', 'BLOOM', 'BERT'], 'data_engineering': ['AWS EC2', 'FireWorks AI', 'Lightning AI', 'Data Pipelines', 'Data Augmentation'], 'cloud_platforms': ['AWS', 'Google Cloud

## Add CV and additional information
The current_cv instance keeps a record of the most recent updated Cover Letter and additional information on work experience, projects, achievements, and certifications. 

### Input arguments:
`resume`=Parsed resume (Dict). \
`cv` = Cover letter text. \
`add_skills` = Additional skills not mentioned in the resume (Dict). \
`add_work_experience` = Additional work experience not mentioned in the resume (Dict). \
`add_education` = Additional education not mentioned in the resume (Dict). \
`add_projects` = Additional Projects not mentioned in the resume (Dict). \
`add_achievements` = Additional achievements not mentioned in the resume (Dict). \
`add_certifications` = Additional certifications not mentioned in the resume (Dict). 

#### additional_information format:
```{``` \
```"information": text,``` \
```"organization": text,``` \
```"date": "dd/mm/yyyy - dd/mm/yyyy" (or) "dd/mm/yyyy"``` \
```}```

The job_listing instance keeps a record of the requirements and description of the job posting. 

In [26]:
print(parsed_resume.keys())

dict_keys(['name', 'contact_info', 'summary', 'skills', 'education', 'research_experience', 'work_experience', 'certifications'])


In [27]:
record = Record(resume=parsed_resume)
with open('job_description.txt', 'r') as f:
    job_description = f.read()

ro_system_prompt = 'You are a resume optimizer. \
Given the current resume(#current_resume) and the job description(#job_description), \
rewrite the current resume to highlight the relevant skills, experience, and quantifiable achievements that match the job description.'

ro_user_prompt = f'Please optimize my current resume \
The current resume is provided under #current_resume \
and job description is provided under #job_description \
\n\t#current_resume: {record.resume} \
\n\t#job_description: {job_description}'

ro_text_worker = TextWorker(client=client, model_id="gpt-4o-mini", sys_prompt=ro_sys_prompt, user_prompt=ro_user_prompt)
new_resume = ro_text_worker.get_output()
print(new_resume)

```json
{
  "current_resume": {
    "name": "Harish Sista",
    "contact_info": {
      "phone": "(201) 310-5683",
      "email": "hsista@stevens.edu",
      "address": "2642 28th St, Apt 4F, Astoria, NY-11102"
    },
    "summary": "Ph.D. Candidate researching in ML and NLP with 7+ years of experience working with ML models, Big Data, and Cloud Computing platforms and 2+ years of experience working in a professional environment. Spearheaded an Evidence Extraction and Fact Verification model using Prompt Engineering and tested it with various LLMs. Invented a Black Box Optimization model for online hyperparameter fine-tuning, significantly enhancing model performance in real-time applications.",
    "skills": {
      "programming_languages": ["Python", "SQL", "C++", "Java", "Swift", "HTML"],
      "ai_tools": ["PyTorch", "HuggingFace", "TensorFlow", "AutoML", "SciKitLearn", "NumPy"],
      "llms": ["GPT", "LLAMA", "BLOOM", "BERT"],
      "data_engineering": ["AWS EC2", "FireWorks AI", 

In [36]:
ra_system_prompt = 'You are a resume analyzer and a question generator.\
                    given the current resume(#current_resume) and the job description(#job_description), \
                    provide keywords from the job description where the resume fails to address the requirements. \
                    Provide the keywords with a question describing how the resume fails to meet this requirement. \
                    Present the output in the following JSON output format: \
                    \n\t{\"keywords\": [{"keyword": system keyword, "question": system keyword question}]}'
ra_user_prompt = f'#current_resume: {record.resume} \
                    #job_description: {job_description}'

ra_text_worker = TextWorker(client=client, model_id="gpt-4o-mini", sys_prompt=ra_system_prompt, user_prompt=ra_user_prompt, json_format=True)
system_insights = ra_text_worker.get_output()
print(system_insights)

{'keywords': [{'keyword': 'AI & GenAI experience', 'question': 'The job description requires 1-3 years of experience working on machine learning or AI projects with a focus on model feature set preparation. How does your experience align with this requirement?'}, {'keyword': 'Data Engineering Expertise', 'question': 'The role demands proven experience in quickly building data pipelines and using large datasets for ML/GenAI applications. Can you elaborate on your specific experience in data engineering, particularly with platforms like Snowflake?'}, {'keyword': 'AutoML familiarity', 'question': 'The position favors familiarity with AutoML tools and automated feature engineering techniques. Can you clarify your exposure to AutoML concepts and tools in your previous work?'}, {'keyword': 'AI in Snowflake', 'question': 'The job mentions that experience using AI tools in Snowflake is a plus. Can you detail any experience you have with Snowflake or similar tools in relation to AI?'}, {'keywor